# FKM Nonlinear - full algorithm step-through
The algorithm follows the document "RICHTLINIE NICHTLINEAR / Rechnerischer Festigkeitsnachweis unter expliziter Erfassung nichtlinearen Werkstoffverformungsverhaltens / Für Bauteile aus Stahl, Stahlguss und Aluminiumknetlegierungen / 1.Auflage, 2019"

The used values are according to "Akademisches Beispiel", chapter 2.7.1.

*Note that this notebook is for demonstration of the algorithm, for actual assessment use other notebooks, e.g., [fkm_nonlinear](fkm_nonlinear.ipynb)*

#### Python module imports

In [ ]:
# standard modules
import pandas as pd
import numpy as np
import itertools
import matplotlib.pyplot as plt
import copy

# pylife
import pylife
import pylife.vmap
import pylife.stress.equistress
import pylife.strength.fkm_load_distribution
import pylife.strength.fkm_nonlinear
import pylife.strength.fkm_nonlinear.damage_calculator
import pylife.strength.damage_parameter
import pylife.strength.woehler_fkm_nonlinear
import pylife.materiallaws
import pylife.stress.rainflow
import pylife.stress.rainflow.recorders
import pylife.stress.rainflow.fkm_nonlinear
import pylife.materiallaws.notch_approximation_law
import pylife.materiallaws.notch_approximation_law_seegerbeste

import pylife.strength.fkm_nonlinear.parameter_calculations as parameter_calculations

## 2.6.1 Collect all input data

#### Material parameters

In [ ]:
assessment_parameters = pd.Series({
        'MatGroupFKM': 'Steel',  # [Steel, SteelCast, Al_wrought] material group
        'FinishingFKM': 'none',  # type of surface finisihing
        'R_m': 600,              # [MPa] ultimate tensile strength (de: Zugfestigkeit)
        #'K_RP': 1,               # [-] surface roughness factor, set to 1 for polished surfaces or determine from the diagrams below
        'R_z': 250,              # [um] average roughness (de: mittlere Rauheit), only required if K_RP is not specified directly
    
        'P_A': 7.2e-5,           # [-] failure probability, set to 0.5 to not consider additional safety (de: auszulegende Ausfallwahrscheinlichkeit)
        # beta: 0.5,             # damage index, specify this as an alternative to P_A
    
        'P_L': 2.5,              # [%] (one of 2.5%, 50%) (de: Auftretenswahrscheinlilchkeit der Lastfolge)
        'c':   3,              # [MPa/N] (de: Übertragungsfaktor Vergleichsspannung zu Referenzlast im Nachweispunkt, c = sigma_I / L_REF)
        'A_sigma': 339.4,        # [mm^2] (de: Hochbeanspruchte Oberfläche des Bauteils)
        'A_ref': 500,            # [mm^2] (de: Hochbeanspruchte Oberfläche eines Referenzvolumens), usually set to 500
        'G': 2/15,               # [mm^-1] (de: bezogener Spannungsgradient)
        's_L': 10,               # [MPa] standard deviation of Gaussian distribution
        'K_p': 3.5,              # [-] (de: Traglastformzahl) K_p = F_plastic / F_yield (3.1.1)
        'r': 15,                 # [mm] radius (?)
})

## Notes on the parameters
### The highly loaded surface $A_\sigma$
The highly loaded surface parameter $A_\sigma$ is needed when the expected fracture starts at the component's surface. It can be computed using the algorithm "SPIEL" as described in the FKM guideline nonlinear. (cf. chapter 3.1.2). For simple common geometries like notched plates and shafts, table 2.6 of the FKM guideline nonlinear presents formulas for $A_\sigma$.

### The relative stress gradient $G$
The relative stress gradient $G$ is usually determined from finite element calculations, but can also be estimated by heuristic, cf. eq. (2.5-34) in the FKM guideline nonlinear.

### The surface roughness factor $K_{R,P}$
The factor of surface roughness can either be manually specified in the ``assessment_parameters`` variable. If it is omitted, the FKM formula is used to estimate it from ultimate tensile strength, $R_m$, and surface roughness, $R_z$.
For polished components, the factor should be set to $K_{R,P}=1$. For other materials, it can be retrieved from the  diagrams in Fig. 2.8 in the FKM guideline nonlinear.

### The safety factor of the component and the failure probability $P_A$
For calculations that should be compared with cyclic experiments, set the failure probability to $P_A=50\%$.
For added safety in assessment concepts, the FKM guideline nonlinear suggests the following failure probabilities ``P_A``:
* Redundant components: $P_A=2.3\cdot 10^{-1}$, $P_A=10^{-3}$, and $P_A=7.2\cdot 10^{-5}$ for moderate, severe, and very severe failure consequences, respectively.
* Non-redundant components: $P_A=10^{-3}$, $P_A=7.2\cdot 10^{-5}$, and $P_A=10^{-5}$ for moderate, severe, and very severe failure consequences, respectively.

You can additionally specify the parameter ``beta`` (de: Zuverlässigkeitsindex) for the material, which then will be used to compute the material safety factor $\gamma_M$. For $P_A=50\%$ set ``gamma_M=1``.

#### Load sequence

In [ ]:
load_sequence = pd.Series([160, -200, 250, -250, 230, 0, 260])  # [N]

fig = plt.figure(figsize=(5,3))
plt.plot(load_sequence, "o-")
plt.ylabel("F")

## Estimate tensile strength $R_m$ from nominal value $R_{m,N}$

There are three possible methods to consider the statistical distribution of the load:
* A normal distribution with given standard deviation, $s_L$
* A logarithmic-normal distribution with given standard deviation $LSD_s$
* An unknown distribution, instead use a constant factor $P_L = 2.5\%$

In [ ]:
# Choose one of the following three lines:

# FKMLoadDistributionNormal, uses assessment_parameters.s_L, assessment_parameters.P_L, assessment_parameters.P_A
scaled_load_sequence = load_sequence.fkm_safety_normal_from_stddev.scaled_load_sequence(assessment_parameters)
gamma_L = load_sequence.fkm_safety_normal_from_stddev.gamma_L(assessment_parameters)
print(f"scaling factor gamma_L: {gamma_L:.5f}")

# FKMLoadDistributionLognormal, uses assessment_parameters.LSD_s, assessment_parameters.P_L, assessment_parameters.P_A
assessment_parameters["LSD_s"] = 1
#scaled_load_sequence = load_sequence.fkm_safety_lognormal_from_stddev.scaled_load_sequence(assessment_parameters)

# FKMLoadDistributionBlanket, uses input_parameters.P_L
#scaled_load_sequence = load_sequence.fkm_safety_blanket.scaled_load_sequence(assessment_parameters)

# 79

Compute load sequence for lifetime assessment, $L = c \cdot \gamma_L \cdot \mathcal{L}$ with $c = \dfrac{\sigma_V}{L_{REF}}$.  
See 2.3.3, 2.3.4

In [ ]:
# scale load sequence by reference load
#c = 1/L_REF
c = assessment_parameters.c
scaled_load_sequence = c * gamma_L * load_sequence
display(scaled_load_sequence)

# plot load sequence
plt.rcParams.update({'font.size': 22})
plt.plot(range(len(scaled_load_sequence)), load_sequence, "o-", c="g", lw=3)
plt.grid()
plt.ylabel(r"$\sigma_V$ [MPa]")

## 2.6.3  Determine cyclic material parameters according to Ramberg-Osgood
This can be done by experiments or by the formulas given in the guideline. This Jupyter notebook uses the implemented formulas. So far, we have specified the following quantities.

In [ ]:
assessment_parameters

In [ ]:
assessment_parameters = pylife.strength.fkm_nonlinear.parameter_calculations.calculate_cyclic_assessment_parameters(assessment_parameters)

## 2.6.4 Estimate material SN-curve (Werkstoff-Wöhlerlinie)

### 2.5.5.1 Determine from experiments
* Conduct strain-driven experiments with $R_\varepsilon = -1$. Record values for $\sigma_a, \varepsilon_{a,\text{ges}}$ and $N_\text{Werkstoff}$.
* Compute $P_{RAM}=\sqrt{\sigma_a\cdot \varepsilon_{a,\text{ges}} \cdot E}$ for every single experiment.
* Use the maximum-likelihood method to infer the parameters $d_1, d_2, P_{RAM,Z,WS}$. For details, refer to the FKM nonlinear document, Sec. 2.5.5.1, p. 40.

### 2.5.5.2 Estimate from formulas
Alternatively, estimate the material SN-curve from the ultimate tensile strength $R_m$.

In [ ]:
# calculate the parameters for the material woehler curve 
# (for both P_RAM and P_RAJ, the variable names do not interfere)
assessment_parameters = parameter_calculations.calculate_material_woehler_parameters_P_RAM(assessment_parameters)
assessment_parameters = parameter_calculations.calculate_material_woehler_parameters_P_RAJ(assessment_parameters)

## 2.6.5 Estimate component SN-curve (Bauteil-Wöhlerlinie)
Consideration of non-local effects on the lifetime of the component.

### 2.6.5.1, 2.5.6.1 Size and geometry factor $n_P$, Spannungsgradient $G$, $A_\sigma$
Compute the factor for non-local influences, $n_P = n_{bm}(R_m, G) \cdot n_{st}(A_\sigma)$, where $n_{bm}$ is the fracture mechanics factor (de: bruchmechanische Stützzahl) and $n_{st}$ is the statistic factor (de: statistische Stützzahl). The factors depend on the stress gradient, $G$, and the highly loaded surface, $A_\sigma$, respectively.

In [ ]:
assessment_parameters = parameter_calculations.calculate_nonlocal_parameters(assessment_parameters)

### 2.6.5.2, 2.5.6.2 Roughness factor $K_{R,P}$
Compute the influence factor of the roughness, $K_{R,P}$, which is estimated based on the ultimate tensile strength, $R_m$, and the surface roughness, $R_z$. See also the diagrams "Abbildung 2.21" show above.

In [ ]:
assessment_parameters = parameter_calculations.calculate_roughness_parameter(assessment_parameters)

In [ ]:
assessment_parameters = parameter_calculations.calculate_failure_probability_factor_P_RAM(assessment_parameters)
assessment_parameters = parameter_calculations.calculate_failure_probability_factor_P_RAJ(assessment_parameters)

### 2.6.5.3, 2.5.6.3 safety factor $\gamma_M$
Compute the factors to derive the component Wöhler curve from the material Wöhler curve. The factors for both P_RAM and P_RAJ are computed.

In [ ]:
assessment_parameters = parameter_calculations.calculate_component_woehler_parameters_P_RAM(assessment_parameters)
assessment_parameters = parameter_calculations.calculate_component_woehler_parameters_P_RAJ(assessment_parameters)

### 2.6.5.4, 2.5.6   formula for component SN-curve

Woehler curve for P_RAM

In [ ]:
component_woehler_curve_parameters = assessment_parameters[["P_RAM_Z", "P_RAM_D", "d_1", "d_2"]]
component_woehler_curve_P_RAM = component_woehler_curve_parameters.woehler_P_RAM

Wöhler curve for P_RAJ

In [ ]:
component_woehler_curve_parameters = assessment_parameters[["P_RAJ_Z", "P_RAJ_D_0", "d_RAJ"]]
component_woehler_curve_P_RAJ = component_woehler_curve_parameters.woehler_P_RAJ

In [ ]:
assessment_parameters

In [ ]:
1/assessment_parameters.d_1, 1/assessment_parameters.d_2, 1/assessment_parameters.d_RAJ

In [ ]:
assessment_parameters[["P_RAM_Z_WS", "P_RAM_D_WS", "d_1", "d_2"]]

In [ ]:
# plot P_RAM material Woehler curve
n_list = np.logspace(2, 7, num=101, endpoint=True, base=10.0)
plt.rcParams.update({'font.size': 24})
fig,ax = plt.subplots(figsize=(8,6))

# for core
material_woehler_curve_parameters = assessment_parameters[["P_RAM_Z_WS", "P_RAM_D_WS", "d_1", "d_2"]]
material_woehler_curve_parameters["P_RAM_Z"] = assessment_parameters["P_RAM_Z_WS"]
material_woehler_curve_parameters["P_RAM_D"] = assessment_parameters["P_RAM_D_WS"]

material_woehler_curve_P_RAM = pylife.strength.woehler_fkm_nonlinear\
    .WoehlerCurvePRAM(material_woehler_curve_parameters)

line = plt.plot(n_list, material_woehler_curve_P_RAM.calc_P_RAM(n_list), "-", lw=4)
plt.plot(1e3, material_woehler_curve_P_RAM.P_RAM_Z, 
         "o", color=line[0].get_color(), markersize=10)
N_D = material_woehler_curve_P_RAM.fatigue_life_limit
plt.plot(N_D, material_woehler_curve_P_RAM.fatigue_strength_limit, 
         "o", color=line[0].get_color(), markersize=10)
plt.annotate(f"$N_D$ = {N_D:.1e}", (N_D, material_woehler_curve_P_RAM.fatigue_strength_limit), 
             textcoords="offset points", xytext=(0,10), color=line[0].get_color())

plt.xscale('log')
plt.yscale('log')
plt.xlabel('N')
plt.ylabel('P_RAM')
plt.title("P_RAM material Woehler curve")
plt.grid(which='both')

In [ ]:
# plot P_RAM component Woehler curve
n_list = np.logspace(2, 7, num=101, endpoint=True, base=10.0)
plt.rcParams.update({'font.size': 24})
fig,ax = plt.subplots(figsize=(8,6))

# for core
component_woehler_curve_parameters = assessment_parameters[["P_RAM_Z", "P_RAM_D", "d_1", "d_2"]]

component_woehler_curve_P_RAM = pylife.strength.woehler_fkm_nonlinear\
    .WoehlerCurvePRAM(component_woehler_curve_parameters)

# material
line = plt.plot(n_list, material_woehler_curve_P_RAM.calc_P_RAM(n_list), "-", lw=4, label="material")
plt.plot(1e3, material_woehler_curve_P_RAM.P_RAM_Z, 
         "o", color=line[0].get_color(), markersize=10)
N_D = material_woehler_curve_P_RAM.fatigue_life_limit
plt.plot(N_D, material_woehler_curve_P_RAM.fatigue_strength_limit, 
         "o", color=line[0].get_color(), markersize=10)
plt.annotate(f"$N_D$ = {N_D:.1e}", (N_D, material_woehler_curve_P_RAM.fatigue_strength_limit), 
             textcoords="offset points", xytext=(0,10), color=line[0].get_color())


# component
line = plt.plot(n_list, component_woehler_curve_P_RAM.calc_P_RAM(n_list), "-", lw=4, label="component")
plt.plot(1e3, component_woehler_curve_P_RAM.P_RAM_Z, 
         "o", color=line[0].get_color(), markersize=10)
N_D = component_woehler_curve_P_RAM.fatigue_life_limit
plt.plot(N_D, component_woehler_curve_P_RAM.fatigue_strength_limit, 
         "o", color=line[0].get_color(), markersize=10)
plt.annotate(f"$N_D$ = {N_D:.1e}", (N_D, component_woehler_curve_P_RAM.fatigue_strength_limit), 
             textcoords="offset points", xytext=(0,10), color=line[0].get_color())

plt.legend(loc="best")
plt.xscale('log')
plt.yscale('log')
plt.xlabel('N')
plt.ylabel('P_RAM')
plt.title("P_RAM component Woehler curve")
plt.grid(which='both')

## 2.6.7, 2.5.8 Perform HCM rainflow counting
    

* 2.6.6, 2.5.7 Compute stresses and strains, classification, PFAD and AST matrices   
* 2.6.7, 2.5.8.1 HCM algorithm, output (S_a, S_m and epsilon_a) for every hysteresis

In [ ]:
# initialize notch approximation law
E, K_prime, n_prime, K_p = assessment_parameters[["E", "K_prime", "n_prime", "K_p"]]
extended_neuber = pylife.materiallaws.notch_approximation_law.ExtendedNeuber(E, K_prime, n_prime, K_p)

load_sequence_list = scaled_load_sequence
print(load_sequence_list)

# create recorder object
recorder = pylife.stress.rainflow.recorders.FKMNonlinearRecorder()

# create detector object
detector = pylife.stress.rainflow.fkm_nonlinear.FKMNonlinearDetector(
    recorder=recorder, notch_approximation_law=extended_neuber
)

# perform HCM algorithm, first run
detector.process(load_sequence_list, flush=False)
detector_1st = copy.deepcopy(detector)

# perform HCM algorithm, second run
detector.process(load_sequence_list, flush=True)


In [ ]:
detector_1st.history()

In [ ]:
# plot resulting stress-strain curve
sampling_parameter = 50    # choose larger for smoother plot
plotting_data = detector_1st.interpolated_stress_strain_data(n_points_per_branch=sampling_parameter)
plotting_data

In [ ]:
primary = plotting_data.loc[~plotting_data.secondary_branch]
secondary = plotting_data.loc[plotting_data.secondary_branch]

fig, axes = plt.subplots(1, 2, figsize=(12,6))
plt.subplots_adjust(wspace=0.4, 
                    hspace=0.4)

# load-time diagram
import matplotlib
matplotlib.rcParams.update({'font.size': 14})
axes[0].plot(load_sequence_list, "o-", lw=3)
axes[0].grid()
axes[0].set_xlabel("t [s]")
axes[0].set_ylabel("L [N]")
axes[0].set_title("Scaled load sequence")

# stress-strain diagram
axes[1].plot(0,0,"ok")
for _, prim in primary.groupby("load_segment"):
    axes[1].plot(prim.strain, prim.stress, "k-", lw=2)
axes[1].plot(secondary.strain, secondary.stress, "k--", lw=1)
axes[1].grid()
axes[1].set_xlabel(r"$\epsilon$")
axes[1].set_ylabel(r"$\sigma$ [MPa]")
axes[1].set_title("Material response")

# stress-strain diagram
plt.rcParams.update({'font.size': 22})
fig, ax = plt.subplots(1, 1, figsize=(4,4))
ax.plot(0,0,"ok")
for _, prim in primary.groupby("load_segment"):
    ax.plot([e*1e2 for e in prim.strain], prim.stress, "k-", lw=3)
ax.plot([e*1e2 for e in secondary.strain], secondary.stress, "k--", lw=1)
ax.grid()
ax.set_xlabel(r"$\epsilon$ [%]")
ax.set_ylabel(r"$\sigma$ [MPa]")

## 2.6.8, 2.5.9 Compute $P_{RAM}$ and damage sum
2.6.8.1, 2.5.9 Mittelspannungsempfindlichkeit

In [ ]:
recorder.collective

In [ ]:
# define damage parameter
damage_parameter = pylife.strength.damage_parameter.P_RAM(recorder.collective, assessment_parameters)
#display(damage_parameter.collective)

# compute the effect of the damage parameter with the woehler curve
damage_calculator = pylife.strength.fkm_nonlinear.damage_calculator\
    .DamageCalculatorPRAM(damage_parameter.collective, component_woehler_curve_P_RAM)

display(damage_calculator.collective)

## 2.6.9, 2.5.10 Safety assessment with $P_{RAM}$
### 2.6.9.1, 2.5.10.1 Infinite life assessment (Dauerfestigkeitsnachweis)

In [ ]:
# Infinite life assessment
is_life_infinite = damage_calculator.is_life_infinite
print(f"Infinite life:                     {is_life_infinite}")

### 2.6.9.3, 2.5.10.3 Finite life assessment (Betriebsfestigkeitsnachweis)

In [ ]:
# finite life assessment
lifetime_n_cycles = damage_calculator.lifetime_n_cycles
lifetime_n_times_load_sequence = damage_calculator.lifetime_n_times_load_sequence

print(f"Number of bearable cycles:         {lifetime_n_cycles:.0f}")
print(f"Number of bearable load sequences: {lifetime_n_times_load_sequence:.0f}")

## Assessment with $P_{RAJ}$

### Repeat HCM algorithm with Seeger-Beste notch approximation

In [ ]:
# initialize notch approximation law
E, K_prime, n_prime, K_p = assessment_parameters[["E", "K_prime", "n_prime", "K_p"]]
seeger_beste = pylife.materiallaws.notch_approximation_law_seegerbeste.SeegerBeste(E, K_prime, n_prime, K_p)

load_sequence_list = scaled_load_sequence
print(load_sequence_list)

# create recorder object
recorder = pylife.stress.rainflow.recorders.FKMNonlinearRecorder()

# create detector object
detector = pylife.stress.rainflow.fkm_nonlinear.FKMNonlinearDetector(
    recorder=recorder, notch_approximation_law=seeger_beste
)

# perform HCM algorithm, first run
detector.process(load_sequence_list, flush=False)

# perform HCM algorithm, second run
detector.process(load_sequence_list, flush=True)

detector.recorder.collective

## 2.9.8, 2.8.9 Compute $P_{RAJ}$ and damage sum

## 2.9.9, 2.8.10 Safety assessment with $P_{RAJ}$

In [ ]:
# define damage parameter
damage_parameter = pylife.strength.damage_parameter.P_RAJ(recorder.collective, assessment_parameters,\
                                                          component_woehler_curve_P_RAJ)
display(damage_parameter.collective)

# compute the effect of the damage parameter with the woehler curve
damage_calculator = pylife.strength.fkm_nonlinear.damage_calculator\
    .DamageCalculatorPRAJ(damage_parameter.collective, assessment_parameters, component_woehler_curve_P_RAJ)

display(damage_calculator.collective)

# show all columns
pd.set_option('display.max_columns', None)
display(damage_calculator.collective)

In [ ]:
print("Tabelle 2.48, stimmt")
display(damage_calculator.collective[["run_index","epsilon_open_alt","epsilon_min_alt_SP", "epsilon_max_alt_SP", "epsilon_min_LF", "epsilon_max_LF"]])

In [ ]:
print("Tabelle 2.49, stimmt")
display(damage_calculator.collective[["run_index", "is_closed_hysteresis", "S_open", "case_name", "epsilon_open_ein", "epsilon_open", "S_close", "P_RAJ", "P_RAJ_D"]])

## Debug crack opening strain of hystereses

In [ ]:
history = detector.history()
history

In [ ]:
# plot resulting stress-strain curve
sampling_parameter = 50    # choose larger for smoother plot
plotting_data = detector.interpolated_stress_strain_data(n_points_per_branch=sampling_parameter)
    
primary = plotting_data.loc[~plotting_data.secondary_branch]
secondary = plotting_data.loc[plotting_data.secondary_branch]

fig, axes = plt.subplots(1, 2, figsize=(12,6))
plt.subplots_adjust(wspace=0.4, 
                    hspace=0.4)

# load-time diagram
import matplotlib
matplotlib.rcParams.update({'font.size': 14})
axes[0].plot(load_sequence_list, "o-", lw=3)
axes[0].grid()
axes[0].set_xlabel("t [s]")
axes[0].set_ylabel("L [N]")
axes[0].set_title("Scaled load sequence")

# stress-strain diagram
axes[1].plot(0,0,"ok")
#for _, run in primary.groupby("run_index"):
for _, prim in primary.groupby("load_segment"):
    axes[1].plot(prim.strain, prim.stress, "k-", lw=2)
for _, run in secondary.groupby("load_segment"):    
    axes[1].plot(run.strain, run.stress, "k--", lw=1)
axes[1].grid()
axes[1].set_xlabel(r"$\epsilon$")
axes[1].set_ylabel(r"$\sigma$ [MPa]")
axes[1].set_title("Material response")

for hysteresis_index, hysteresis in plotting_data.groupby("hyst_index"):
    if hysteresis_index < 0:
        continue
    fig, ax = plt.subplots(figsize=(12,6))

    primary = hysteresis.loc[~hysteresis.secondary_branch]
    secondary = hysteresis.loc[hysteresis.secondary_branch]
    run_index = hysteresis.run_index.iloc[0]

    hyst_start_segment = history.query(f"hyst_to == {hysteresis_index}").index.get_level_values("load_segment")[0]
    history_plot = plotting_data[plotting_data.load_segment < hyst_start_segment]
        
    ax.set_title(f"Hysteresis {hysteresis_index}, Run {run_index}")
    for _, segment in history_plot.groupby("load_segment"):
        ax.plot(segment.strain, segment.stress, "gray", lw=1)
    for _, prim in primary.groupby("load_segment"):
        ax.plot(prim.strain, prim.stress, "b-", lw=2)
    ax.plot(secondary.strain, secondary.stress, "b--", lw=1)
    ax.grid()
    ax.set_xlabel(r"$\epsilon$")
    ax.set_ylabel(r"$\sigma$ [MPa]")
    
    

In [ ]:
# plot all hystereses
# get all graph data
sampling_parameter = 50    # choose larger for smoother plot
plotting_data = detector.interpolated_stress_strain_data(n_points_per_branch=sampling_parameter)

primary = plotting_data.loc[~plotting_data.secondary_branch]
secondary = plotting_data.loc[plotting_data.secondary_branch]

fig, ax = plt.subplots(figsize=(12,6))

for _, prim in primary.groupby("load_segment"):
    ax.plot(prim.strain, prim.stress, "b-", lw=2)
for _, sec in secondary.groupby("load_segment"):
    ax.plot(sec.strain, sec.stress, "b--", lw=1)

#ax.plot(primary.strain, primary.stress, "b-", lw=2)
#ax.plot(secondary.strain, secondary.stress, "b--", lw=1)
ax.grid()
ax.set_xlabel(r"$\epsilon$")
ax.set_ylabel(r"$\sigma$ [MPa]")
ax.set_title("All hystereses")


collective = damage_parameter.collective

# get graph data of only hystereses
sampling_parameter = 50    # choose larger for smoother plot

for hysteresis_index, hysteresis in plotting_data.query("hyst_index >= 0").groupby("hyst_index"):
    fig, ax = plt.subplots(figsize=(12,6))
    
    primary = hysteresis.loc[~hysteresis.secondary_branch]
    secondary = hysteresis.loc[hysteresis.secondary_branch]
    run_index = hysteresis.run_index.iloc[0]   

    hyst_start_segment = history.query(f"hyst_to == {hysteresis_index}").index.get_level_values("load_segment")[0]
    history_plot = plotting_data[plotting_data.load_segment < hyst_start_segment]
        
    
    ax.set_title(f"Hysteresis {hysteresis_index}, Run {run_index}")
    for _, segment in history_plot.groupby("load_segment"):
        ax.plot(segment.strain, segment.stress, "gray", lw=1)        
    for _, prim in primary.groupby("load_segment"):
        ax.plot(prim.strain, prim.stress, "b-", lw=2)    
    ax.plot(secondary.strain, secondary.stress, "b--", lw=1)
    
    # plot crack opening points
    S_open = collective.iloc[hysteresis_index, collective.columns.get_loc("S_open")]
    S_close = collective.iloc[hysteresis_index, collective.columns.get_loc("S_close")]
    epsilon_open = collective.iloc[hysteresis_index, collective.columns.get_loc("epsilon_open")]
    epsilon_open_ein = collective.iloc[hysteresis_index, collective.columns.get_loc("epsilon_open_ein")]
    
    ax.plot(epsilon_open_ein, S_open, "+", markersize=20, markeredgewidth=4, color="b", label="ε_open,ein")
    ax.plot(epsilon_open_ein, S_close, "+", markersize=20, markeredgewidth=4, color="b")
    
    ax.plot(epsilon_open, S_open, "+", markersize=20, markeredgewidth=2, color="m", label="ε_open")
    ax.plot(epsilon_open, S_close, "+", markersize=20, markeredgewidth=2, color="m")
    ax.plot([epsilon_open, epsilon_open], 
            [-400,400], "-", color="m")
    
    # plot epsilons
    S_min = collective.iloc[hysteresis_index, collective.columns.get_loc("S_min")]
    S_max = collective.iloc[hysteresis_index, collective.columns.get_loc("S_max")]
    
    epsilon_min_alt_SP = collective.iloc[hysteresis_index, collective.columns.get_loc("epsilon_min_alt_SP")]
    epsilon_max_alt_SP = collective.iloc[hysteresis_index, collective.columns.get_loc("epsilon_max_alt_SP")]
    epsilon_min_LF = collective.iloc[hysteresis_index, collective.columns.get_loc("epsilon_min_LF")]
    epsilon_max_LF = collective.iloc[hysteresis_index, collective.columns.get_loc("epsilon_max_LF")]
    
    ax.plot([epsilon_min_alt_SP,epsilon_min_alt_SP], [S_min*0.9,S_max*1.1], ":", label="ε_min,alt,SP")
    ax.plot([epsilon_max_alt_SP,epsilon_max_alt_SP], [S_min*0.9,S_max*1.1], "-", label="ε_max,alt,SP")
    ax.plot([epsilon_min_LF,epsilon_min_LF], [S_min,S_max], ":", label="ε_min,LF")
    ax.plot([epsilon_max_LF,epsilon_max_LF], [S_min,S_max], "-", label="ε_max,LF")
    
    ax.grid()
    ax.legend(bbox_to_anchor=(1.1,1), loc="upper left")
    ax.set_xlabel(r"$\epsilon$")
    ax.set_ylabel(r"$\sigma$ [MPa]")
    


### 2.9.9.1, 2.8.10.1 Infinite life assessment (Dauerfestigkeitsnachweis)

In [ ]:
# Infinite life assessment
is_life_infinite = damage_calculator.is_life_infinite
print(f"Infinite life:                     {is_life_infinite}")

### 2.9.8.4, 2.8.10.2 Finite life assessment (Betriebsfestigkeitsnachweis)

In [ ]:
# finite life assessment
lifetime_n_cycles = damage_calculator.lifetime_n_cycles
lifetime_n_times_load_sequence = damage_calculator.lifetime_n_times_load_sequence

print(f"Number of bearable cycles:         {lifetime_n_cycles:.0f}")
print(f"Number of bearable load sequences: {lifetime_n_times_load_sequence:.0f}")